![](2024-08-14-18-07-11.png)

# IMAGE SEGMENTATION 
## Author: Diversa
## Last Update: 22/8/2024
## Proyect: Feminist Urban Sense
## Contact: hello@diversa.studio
---

# Image Segmentation

## Objectives
1. **Implement a YOLO-based model for image segmentation:** This notebook aims to detect and classify various categories related to safety, proximity, diversity, autonomy, vitality, and representativity using the YOLOWorld model.

## Methodology

### 1. Environment Preparation
- **Library Installation:** The necessary libraries, including `inference-gpu[yolo-world]` and `supervision`, are installed to support the YOLO model implementation.

### 2. Model Configuration
- **YOLOWorld Model:** The notebook configures the YOLOWorld model using the `yolo_world/l` model ID for segmentation tasks.

### 3. Image Segmentation Categories
- **Category Definitions:** The notebook defines six key categories for segmentation:
  - **Safety:** Includes items like "Light poles" and "Streetlights."
  - **Proximity:** Includes items like "Bus stop" and "Transit stop."
  - **Diversity:** Includes items like "Playground" and "Jungle gym."
  - **Autonomy:** Includes items like "Bench" and "Outdoor bench."
  - **Vitality:** Includes items like "Playground" and "Play structure."
  - **Representativity:** Includes items like "Police station" and "Precinct."

- **Segmentation Process:** For each category, images are processed using the defined YOLO model to detect and classify relevant objects.

### 4. Data Analysis
- **CSV Data Handling:** The results of the segmentation are stored in CSV files, which are then analyzed to extract insights related to the different categories.
- **Category-wise Analysis:** The notebook provides a detailed analysis of the results for each category, using the processed data stored in CSV files.

## Important Aspects
- **Comprehensive Segmentation:** This notebook effectively segments and classifies images into predefined categories using a state-of-the-art YOLO model.
- **Scalability:** The methods used can be scaled to handle larger datasets or different categories.
- **Documentation:** Each step is well-documented to ensure clarity and reproducibility.


### 1.- Enviroment Preparation
Instalations and Imports of libraries

In [ ]:
!pip install -q inference-gpu[yolo-world]==0.9.12rc1
!pip install -q supervision==0.19.0rc3


import cv2
import pandas as pd
import supervision as sv
import os
from pathlib import Path

from tqdm import tqdm
from inference.models.yolo_world.yolo_world import YOLOWorld

# Se traen las librerias del google para realizar la conexion con el drive
from google.colab import files
from google.colab import drive
drive.mount("/content/drive") # Se realiza la peticion para conexion a la cuenta de google drive

### 2.- Model Values Configuration

In [ ]:
model = YOLOWorld(model_id="yolo_world/l")

In [ ]:
def procesar_imagenes_yolo(path_origen, path_destino):
    if not os.path.exists(path_destino):
        os.makedirs(path_destino)

    detecciones_lista = []
    summary_list = []

    imagenes = [f for f in os.listdir(path_origen) if os.path.isfile(os.path.join(path_origen, f))]

    for imagen in imagenes:
        imagen_path = os.path.join(path_origen, imagen)
        image = cv2.imread(imagen_path)

        if image is None:
            print(f"Error, image: '{image}' was not valid and will be omited")
            continue

        results = model.infer(image)
        detections = sv.Detections.from_inference(results)

        BOUNDING_BOX_ANNOTATOR = sv.BoundingBoxAnnotator(thickness=2)
        LABEL_ANNOTATOR = sv.LabelAnnotator(text_thickness=2, text_scale=1, text_color=sv.Color.BLACK)
        annotated_image = image.copy()
        annotated_image = BOUNDING_BOX_ANNOTATOR.annotate(annotated_image, detections)
        annotated_image = LABEL_ANNOTATOR.annotate(annotated_image, detections)

        imagen_destino_path = os.path.join(path_destino, imagen)
        cv2.imwrite(imagen_destino_path, annotated_image)

        detected_tags = []
        for detection in detections:
            x1, y1, x2, y2, confianza, etiqueta = detection
            detecciones_lista.append({
                'imagen': imagen,
                'x1': x1,
                'y1': y1,
                'x2': x2,
                'y2': y2,
                'confianza': confianza,
                'etiqueta': etiqueta
            })
            # Assuming 'etiqueta' is a dictionary, let's extract the label
            if isinstance(etiqueta, dict):
                # Replace 'label' with the actual key that contains the label in your etiqueta dictionary
                label = etiqueta.get('label', str(etiqueta))
            else:
                label = str(etiqueta)
            detected_tags.append(label)

        summary_list.append({
            'imagen': imagen,
            'detected': ', '.join(set(detected_tags))  # Use set to remove duplicates
        })

    sv.plot_image(annotated_image, (10, 10))

    # Create and save the detailed DataFrame
    df_detecciones = pd.DataFrame(detecciones_lista)
    df_detecciones.to_csv(os.path.join(path_destino, 'ObjectDetected.csv'), index=False)

    # Create and save the summary DataFrame
    df_summary = pd.DataFrame(summary_list)
    df_summary.to_csv(os.path.join(path_destino, 'DetectionSummary.csv'), index=False)

    return df_detecciones, df_summary

### 3.- Images Segmentation Categories

#### **SAFETY CATEGORIE**

In [ ]:
pathOrigen = r"'your_paths'"
path_destino = r"'your_paths'"

In [ ]:
model = YOLOWorld(model_id="yolo_world/l")
safety_classes = [
    "Light poles", "Streetlights,", "Lamp posts",
    "Neon light", "Fluorescent light,", "LED sign",
    "White light", "Halogen light,", "Daylight bulb",
    "Yellow light", "Amber light,", "Sodium vapor light",
    "Signage", "Directional sign,", "Traffic sign",
    "CCTV", "Surveillance camera,", "Security camera",
    "Marked crosswalk", "Zebra crossing,", "Pedestrian crossing",
    "Ramps", "Incline plane,", "Access ramp",
    "Handrail", "Guardrail,", "Safety rail",
    "Stair handrail", "Balustrade,", "Railing",
    "Elevator/Lift", "Escalator,", "Stair lift",
    "Bush", "Shrub,", "Hedge",
    "Trees", "Saplings,", "Shade trees",
    "Maps", "Wayfinding map,", "City map",
    "Pedestrian bridge", "Footbridge,", "Walkway bridge",
    "Overpass", "Flyover,", "Pedestrian overpass"
]
model.set_classes(safety_classes)

In [ ]:
df_resultados = procesar_imagenes_yolo(pathOrigen, path_destino)

#### **PROXIMITY CATEGORIE**

In [ ]:
pathOrigen = r"'your_paths'"
path_destino = r"'your_paths'"

In [ ]:
model = YOLOWorld(model_id="yolo_world/l")
proximity_classes = [
    "Bus stop", "Transit stop", "Bus shelter",
    "Metro stop", "Subway station", "Underground station",
    "Bicycle lines", "Bike path", "Cycle lane",
    "Bicycle parking", "Bike rack", "Cycle stand",
    "Bicycle storage cage", "Bike locker", "Bicycle enclosure",
    "Public toilet", "Restroom", "Lavatory",
    "Local stores", "Corner shop", "Convenience store",
    "Vendor stall", "Market stall", "Kiosk",
    "Kiosks", "Booth", "Newsstand",
    "Bus stop bench", "Waiting bench", "Transit seating",
    "Bus stop roof", "Canopy", "Shelter roof",
    "Ramp handrail", "Accessibility rail", "Incline handrail",
    "Stair handrail", "Guardrail", "Banister"
]


model.set_classes(proximity_classes)

In [ ]:
df_resultados = procesar_imagenes_yolo(pathOrigen, path_destino)

#### **DIVERSITY CATEGORIE**

In [ ]:
pathOrigen = r"'your_paths'"
path_destino = r"'your_paths'"

In [ ]:
model = YOLOWorld(model_id="yolo_world/l")
diversity_classes = [
    "Playground", "Play area", "Jungle gym",
    "Bench", "Park bench", "Garden seat",
    "Seats", "Chair", "Stool",
    "Table", "Dining table", "Picnic table",
    "Coffee table", "Side table", "Accent table",
    "Automatic teller machine", "ATM", "Cash machine",
    "Payphone cabinets", "Phone booth", "Telephone kiosk",
    "Parking meters", "Ticket machine", "Metered parking device",
    "Vending and ticket machines", "Dispenser", "Ticket kiosk"
]
model.set_classes(diversity_classes)

In [ ]:
df_resultados = procesar_imagenes_yolo(pathOrigen, path_destino)

#### **AUTONOMY CATEGORIE**

In [ ]:
pathOrigen = r"'your_paths'"
path_destino = r"'your_paths'"

In [ ]:
model = YOLOWorld(model_id="yolo_world/l")
autonomy_classes = [
    "Bench", "Park seat", "Outdoor bench",
    "Seats", "Armchair", "Lounge chair",
    "Table", "Picnic bench", "Patio table",
    "Coffee table", "Tea table", "Cocktail table",
    "Shelter", "Pavilion", "Gazebo",
    "Sun umbrella", "Parasol", "Shade umbrella",
    "Grassy", "Turf", "Lawn",
    "Sandbox", "Sandpit", "Play pit",
    "Cement box", "Concrete planter", "Stone container",
    "Wood", "Timber", "Lumber",
    "Metal box", "Steel container", "Aluminum box",
    "Concrete bollards", "Stone bollard", "Traffic bollard",
    "Metal bollards", "Steel post", "Iron bollard",
    "Ramps", "Incline", "Gradient",
    "Stair handrail", "Baluster", "Hand grip",
    "Elevator/Lift", "Platform lift", "Hoist"
]

model.set_classes(autonomy_classes)

In [ ]:
df_resultados = procesar_imagenes_yolo(pathOrigen, path_destino)

#### **VITALITY CATEGORIE**

In [ ]:
pathOrigen = r"'your_paths'"
path_destino = r"'your_paths'"

In [ ]:
model = YOLOWorld(model_id="yolo_world/l")
vitality_classes = [
    "Playground", "Play structure", "Swing set",
    "Bench", "Garden bench", "Outdoor seat",
    "Table", "Picnic bench", "Coffee table",
    "Parks", "Public garden", "Green space",
    "Fountains", "Water feature", "Water spout",
    "Skate", "Skateboard ramp", "Skate park",
    "Busker", "Street performer", "Musician"
]

model.set_classes(vitality_classes)

In [ ]:
df_resultados = procesar_imagenes_yolo(pathOrigen, path_destino)

#### **REPRESENTATIVITY CATEGORIE**

In [ ]:
pathOrigen = r"'your_paths'"
path_destino = r"'your_paths'"

In [ ]:
model = YOLOWorld(model_id="yolo_world/l")
representativity_classes = [
    "Police station", "Precinct", "Law enforcement office",
    "Kiosks", "Information booth", "Concession stand",
    "Vendor stall", "Market booth", "Food stall",
    "Post boxes", "Mailbox", "Post office box",
    "Graffiti", "Street art", "Wall mural",
    "Mural", "Fresco", "Wall painting",
    "Statue", "Monument", "Sculpture",
    "Sculpture", "Carving", "Figurine"
]

model.set_classes(representativity_classes)

In [ ]:
df_resultados = procesar_imagenes_yolo(pathOrigen, path_destino)

### 4.- Data Analisis

In [ ]:
import pandas as pd
import os

# Define the list of directories where your CSV files are located
paths = ['your_paths']  # Add your paths here

# Function to unify detections in a single cell by 'imagen'
def unify_detections(csv_file):
    df = pd.read_csv(csv_file)

    # Fill NaN values with empty strings
    df['detected'] = df['detected'].fillna('')

    # Group by 'imagen' and create a list of non-empty detections
    df = df.groupby('imagen')['detected'].agg(lambda x: list(filter(None, x))).reset_index()

    return df
# Process all CSV files in the specified paths
for path in paths:
    if os.path.isfile(path) and path.endswith(".csv"):
        df_modified = unify_detections(path)
        directory, filename = os.path.split(path)
        output_file_path = os.path.join(directory, f"modified_{filename}")
        df_modified.to_csv(output_file_path, index=False)

print("All specified CSV files have been processed and saved with 'modified_' prefix.")


In [ ]:
import pandas as pd
import os
import ast

# Define the categories and their corresponding file paths
categories = {'your_paths'}

# Read the original CSV file
original_csv_path = '/content/drive/MyDrive/image_results/autonomy/modified_DetectionSummary.csv'
original_df = pd.read_csv(original_csv_path)

# Initialize a dictionary to store the combined data
combined_data = {row['imagen']: {'autonomy': ast.literal_eval(row['detected'])} for _, row in original_df.iterrows()}

# Process each category CSV file
for category, path in categories.items():
    df = pd.read_csv(path)

    # Add detected columns to combined_data
    for index, row in df.iterrows():
        imagen = row['imagen']
        detected = ast.literal_eval(row['detected'])

        if imagen not in combined_data:
            combined_data[imagen] = {}

        if category not in combined_data[imagen]:
            combined_data[imagen][category] = []

        # Only add non-empty detections
        if detected and detected != []:
            combined_data[imagen][category].extend(detected)

# Create a new DataFrame from the combined data
combined_df = pd.DataFrame(list(combined_data.items()), columns=['imagen', 'detections'])

# Initialize the final DataFrame with the original data
final_df = original_df.copy()

# Add the detection columns and counts to the final DataFrame
for category in ['autonomy'] + list(categories.keys()):
    final_df[f'detected_{category}'] = final_df['imagen'].map(lambda x: combined_df.set_index('imagen').at[x, 'detections'].get(category, []) if x in combined_df.set_index('imagen').index else [])
    final_df[f'count_detected_{category}'] = final_df[f'detected_{category}'].map(len)

# Combine all detections into the original 'detected' column and add the total count
final_df['detected'] = final_df.apply(lambda row: sum([row[f'detected_{cat}'] for cat in ['autonomy'] + list(categories.keys())], []), axis=1)
final_df['count_detected'] = final_df['detected'].map(len)

# Save the final DataFrame to a new CSV file
output_csv_path = '/content/drive/MyDrive/image_results/finalcsv.csv'
final_df.to_csv(output_csv_path, index=False)

print("All category CSV files have been combined and saved with the detection counts.")




In [ ]:
import pandas as pd
import os
import ast

# Define the categories and their corresponding file paths
categories = {'your_paths'}

# Read the original CSV file
original_csv_path = '/mnt/data/DetectionSummary (3).csv'
original_df = pd.read_csv(original_csv_path)

# Initialize a dictionary to store the combined data
combined_data = {}

# Process each category CSV file
for category, path in categories.items():
    df = pd.read_csv(path)

    # Add detected columns to combined_data
    for index, row in df.iterrows():
        imagen = row['imagen']
        detected = row['detected']

        if imagen not in combined_data:
            combined_data[imagen] = {}

        if category not in combined_data[imagen]:
            combined_data[imagen][category] = []

        # Only add non-empty detections
        if detected and detected != '[]':
            combined_data[imagen][category].append(detected)

# Create a new DataFrame from the combined data
combined_df = pd.DataFrame(list(combined_data.items()), columns=['imagen', 'detections'])

# Initialize the final DataFrame with the original data
final_df = original_df.copy()

# Add the detection columns and counts to the final DataFrame
for category in categories.keys():
    final_df[f'detected_{category}'] = final_df['imagen'].map(lambda x: combined_df.set_index('imagen').at[x, 'detections'][category] if x in combined_df.set_index('imagen').index and category in combined_df.set_index('imagen').at[x, 'detections'] else [])
    final_df[f'count_detected_{category}'] = final_df[f'detected_{category}'].map(lambda x: len(x))

# Combine all detections into the original 'detected' column and add the total count
final_df['detected'] = final_df.apply(lambda row: [det for cat in categories.keys() for det in row[f'detected_{cat}']], axis=1)
final_df['count_detected'] = final_df['detected'].map(lambda x: len([i for i in x if i and i != '[]']))

# Save the final DataFrame to a new CSV file
output_csv_path = '/mnt/data/final_combined_detection_summary.csv'
final_df.to_csv(output_csv_path, index=False)

print("All category CSV files have been combined and saved with the detection counts.")
